In [1]:
import spacy

In [40]:
import gensim

In [44]:
import numpy as np
import pandas as pd
from collections import Counter

import gensim.downloader as api

import time

In [2]:
nlp = spacy.load('en_core_web_sm')

In [13]:
df = pd.read_csv('data/aapl_articles.csv')
df.set_index('Unnamed: 0',inplace=True)
df.index.rename('adjusted_date',inplace=True)

In [45]:
t0 = time.time()

# Import word vectors into "model." from Lession 8.05, GA
wv = api.load('word2vec-google-news-300')

print(time.time() - t0)

[==================================================] 100.0% 1662.8/1662.8MB downloaded
375.6538109779358


In [16]:
df.head(1)

,date,title,text,new_date,day,Open,High,Low,Close,Volume,Dividends,Stock Splits
adjusted_date,,,,,,,,,,,,
2020-07-31,"Aug. 3, 2020 4:02 PM ET",Microsoft and Apple take Nasdaq to another all...,The market day started and ended with megacap...,2020-08-03,0,102.7,106.22,100.64,106.07,374336800,0.0,0.0


In [18]:
sample_text = df['text'][0]

' The market day started and ended with megacaps Microsoft and Apple calling the tune as stocks finished higher The Nasdaq closed up 15 to a record high of 1090280 eclipsing the previous closing high of 1076709 hit on July 20 Nasdaq back at record levels  The SP finished up 07 and the Dow ended up 09 Microsoft NASDAQMSFT popped higher as enthusiasm built for its potential acquisition of TikTok where at present its the only suitor Apple NASDAQAAPL added to its stunning Friday gains The other three of the Fab 5 weakened and ended in the red Together those stocks helped the SPDR Information Technology Sector ETF NYSEARCAXLK jump 25 Take tech out and the market performance was pretty even between gaining and losing sectors  The fiscal stimulus negotiations ended today without any progress House Speaker Nancy Pelosi says Democrats will meet with White House officials again tomorrow Senate Majority Leader Mitch McConnell says Democrats are refusing to budge Under Armour was the weakest stock

In [19]:
sample_doc = nlp(sample_text)

In [22]:
print([token.text for token in sample_doc])

[' ', 'The', 'market', 'day', 'started', 'and', 'ended', 'with', 'megacaps', 'Microsoft', 'and', 'Apple', 'calling', 'the', 'tune', 'as', 'stocks', 'finished', 'higher', 'The', 'Nasdaq', 'closed', 'up', '15', 'to', 'a', 'record', 'high', 'of', '1090280', 'eclipsing', 'the', 'previous', 'closing', 'high', 'of', '1076709', 'hit', 'on', 'July', '20', 'Nasdaq', 'back', 'at', 'record', 'levels', ' ', 'The', 'SP', 'finished', 'up', '07', 'and', 'the', 'Dow', 'ended', 'up', '09', 'Microsoft', 'NASDAQMSFT', 'popped', 'higher', 'as', 'enthusiasm', 'built', 'for', 'its', 'potential', 'acquisition', 'of', 'TikTok', 'where', 'at', 'present', 'its', 'the', 'only', 'suitor', 'Apple', 'NASDAQAAPL', 'added', 'to', 'its', 'stunning', 'Friday', 'gains', 'The', 'other', 'three', 'of', 'the', 'Fab', '5', 'weakened', 'and', 'ended', 'in', 'the', 'red', 'Together', 'those', 'stocks', 'helped', 'the', 'SPDR', 'Information', 'Technology', 'Sector', 'ETF', 'NYSEARCAXLK', 'jump', '25', 'Take', 'tech', 'out', 'a

In [23]:
sentences = list(sample_doc.sents)

In [25]:
for sentence in sentences:
    print(sentence)
    print()

 The market day started and ended with megacaps Microsoft and Apple calling the tune as stocks finished higher The Nasdaq closed up 15 to a record high of 1090280 eclipsing the previous closing high of 1076709 hit on July 20 Nasdaq back at record levels  

The SP finished up 07 and the Dow ended up 09 Microsoft NASDAQMSFT popped higher as enthusiasm built for its potential acquisition of TikTok where at present its the only suitor Apple NASDAQAAPL added to its stunning Friday gains

The other three of the Fab 5 weakened and ended in the red

Together those stocks helped the SPDR Information Technology Sector ETF NYSEARCAXLK jump 25

Take tech out and

the market performance was pretty even between gaining and losing sectors  

The fiscal stimulus negotiations ended today without

any progress House Speaker Nancy Pelosi says Democrats will meet with White House officials again

tomorrow Senate Majority Leader Mitch McConnell says Democrats are refusing to budge Under Armour was the weak

In [26]:
for token in sample_doc:
    print(token, token.idx)

  0
The 1
market 5
day 12
started 16
and 24
ended 28
with 34
megacaps 39
Microsoft 48
and 58
Apple 62
calling 68
the 76
tune 80
as 85
stocks 88
finished 95
higher 104
The 111
Nasdaq 115
closed 122
up 129
15 132
to 135
a 138
record 140
high 147
of 152
1090280 155
eclipsing 163
the 173
previous 177
closing 186
high 194
of 199
1076709 202
hit 210
on 214
July 217
20 222
Nasdaq 225
back 232
at 237
record 240
levels 247
  254
The 255
SP 259
finished 262
up 271
07 274
and 277
the 281
Dow 285
ended 289
up 295
09 298
Microsoft 301
NASDAQMSFT 311
popped 322
higher 329
as 336
enthusiasm 339
built 350
for 356
its 360
potential 364
acquisition 374
of 386
TikTok 389
where 396
at 402
present 405
its 413
the 417
only 421
suitor 426
Apple 433
NASDAQAAPL 439
added 450
to 456
its 459
stunning 463
Friday 472
gains 479
The 485
other 489
three 495
of 501
the 504
Fab 508
5 512
weakened 514
and 523
ended 527
in 533
the 536
red 540
Together 544
those 553
stocks 559
helped 566
the 573
SPDR 577
Information 582
T

In [28]:
stopwords = spacy.lang.en.stop_words.STOP_WORDS

In [29]:
len(stopwords)

326

In [30]:
words = [token.text for token in sample_doc if not token.is_stop and not token.is_punct]

In [35]:
word_freq = Counter(words)

In [37]:
def is_token_allowed(token):
    '''
        Only allow valid tokens which are not stop words
        and punctuation symbols.
    '''
    if (not token or not token.string.strip() or
        token.is_stop or token.is_punct):
        return False
    return True

def preprocess_token(token):
    # Reduce token to its lowercase lemma form
    return token.lemma_.strip().lower()

In [38]:
complete_filtered_tokens = [preprocess_token(token)
    for token in sample_doc if is_token_allowed(token)]
complete_filtered_tokens

['market',
 'day',
 'start',
 'end',
 'megacap',
 'microsoft',
 'apple',
 'call',
 'tune',
 'stock',
 'finish',
 'high',
 'nasdaq',
 'close',
 '15',
 'record',
 'high',
 '1090280',
 'eclipse',
 'previous',
 'closing',
 'high',
 '1076709',
 'hit',
 'july',
 '20',
 'nasdaq',
 'record',
 'level',
 'sp',
 'finish',
 '07',
 'dow',
 'end',
 '09',
 'microsoft',
 'nasdaqmsft',
 'pop',
 'higher',
 'enthusiasm',
 'build',
 'potential',
 'acquisition',
 'tiktok',
 'present',
 'suitor',
 'apple',
 'nasdaqaapl',
 'add',
 'stunning',
 'friday',
 'gain',
 'fab',
 '5',
 'weaken',
 'end',
 'red',
 'stock',
 'help',
 'spdr',
 'information',
 'technology',
 'sector',
 'etf',
 'nysearcaxlk',
 'jump',
 '25',
 'tech',
 'market',
 'performance',
 'pretty',
 'gain',
 'lose',
 'sector',
 'fiscal',
 'stimulus',
 'negotiation',
 'end',
 'today',
 'progress',
 'house',
 'speaker',
 'nancy',
 'pelosi',
 'say',
 'democrats',
 'meet',
 'white',
 'house',
 'official',
 'tomorrow',
 'senate',
 'majority',
 'leader',
 